In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
from scipy.stats import t
from scipy.stats import norm
from scipy.stats import gamma

$x_i|\mu\sim N(\mu,\sigma^2)\\
\mu\sim N(\mu_0,\sigma_0^2) \Rightarrow \mu|x_1,\ldots,x_n\sim N\left(\frac{\mu_0\sigma^2+n\bar{x}\sigma_0^2}{\sigma^2+n\sigma_0^2},\left(\frac{1}{\sigma_0^2}+\frac{n}{\sigma^2}\right)^{-1} \right)$

In [3]:
# posterior
def posterior_norm_param(m0, S0, S, X):
    n = len(X)
    Sp = 1/(1/(S0*S0)+n/(S*S))
    a = Sp*(m0/S0/S0+np.sum(X)/S/S)
    return np.array((a,np.sqrt(Sp)))

In [151]:
# posterior for Cauchy case
def posterior_norm_param(m0, S0, S, X):
    n = len(X)
    Sp = 1/(1/(S0*S0)+n/(S*S))
    a = Sp*(m0/S0/S0+n*np.median(X)/S/S)
    return np.array((a,np.sqrt(Sp)))

In [4]:
# priors
# fact
m = 480
S = 100
# first expert
m1 = 500
S1 = 20
# second expert
m2 = 300
S2 = 100

In [59]:
X1 = norm.rvs(m1,S1,10)
X2 = norm.rvs(m2,S2,10)
X = np.hstack((X1,X2))
cat1 = [1 for x in range(10)]
cat2 = [2 for x in range(10)]
dat = {}
dat['X'] = X
dat['cat'] = np.concatenate((cat1,cat2))
dat

{'X': array([ 501.71514121,  496.66564079,  523.66757932,  528.71121898,
         485.68387459,  479.56885505,  506.77929712,  517.80098719,
         494.20119284,  503.22976228,  327.92470656,  155.55671982,
         348.52653458,   88.65803576,  391.06581166,  305.70778063,
         230.20701031,  235.7491126 ,  307.42515817,  257.06969468]),
 'cat': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])}

In [5]:
# simulated prior probabilities for two experts
X1 = norm.rvs(m1,S1,10000)
X2 = norm.rvs(m2,S2,10000)
plt.hist(X1, bins='auto', alpha=0.5, normed = 1)
plt.hist(X2, bins='auto', alpha=0.5, normed = 1)
lb = min(norm.interval(0.99,m1,S1)[0], norm.interval(0.99,m2,S2)[0])
rb = max(norm.interval(0.99,m1,S1)[1], norm.interval(0.99,m2,S2)[1])
x = np.linspace(lb,rb, 100)
plt.plot(x,norm.pdf(x,m1,S1),linewidth = 4,alpha=0.5)
plt.plot(x,norm.pdf(x,m2,S2),linewidth = 4,alpha=0.5)
plt.show()

In [56]:
N = 100
rng = np.arange(0,N,1)
M1, M2, D1, D2 = np.empty([N]), np.empty([N]), np.empty([N]), np.empty([N]) 
for n in rng:
    X = norm.rvs(m, S, n)
    M1[n], D1[n] = posterior_norm_param(m1, S1, S, X)
    M2[n], D2[n] = posterior_norm_param(m2, S2, S, X)

plt.plot(rng, M1, color='green', alpha = 0.5)
plt.fill_between(rng, M1 + D1, M1 - D1, color='green', alpha=0.5)
plt.plot(rng, M2, color='red', alpha = 0.5, linestyle='--')
plt.fill_between(rng, M2 + D2, M2 - D2, color='red', alpha=0.5)
plt.show()

In [158]:
# posterior
def posterior_norm(N, m0, S0, S, X):
    n = len(X)
    Sp = 1/(1/(S0*S0)+n/(S*S))
    a = Sp*(m0/S0/S0+np.sum(X)/S/S)
    return norm.rvs(a,np.sqrt(Sp),N)

In [163]:
# posterior (for Cauchy case)
def posterior_norm(N, m0, S0, S, X):
    n = len(X)
    Sp = 1/(1/(S0*S0)+n/(S*S))
    a = Sp*(m0/S0/S0+n*np.median(X)/S/S)
    return norm.rvs(a,np.sqrt(Sp),N)

In [164]:
# evidence
#X = norm.rvs(m,S,10000)
X = t.rvs(1,m,S,10000)

In [165]:
# posteriors
lb = min(norm.interval(0.99,m1,S1)[0], norm.interval(0.99,m2,S2)[0])
rb = max(norm.interval(0.99,m1,S1)[1], norm.interval(0.99,m2,S2)[1])
x = np.linspace(lb,rb, 100)
Xp1 = posterior_norm(10000,m1,S1,S,X)
Xp2 = posterior_norm(10000,m2,S2,S,X)
plt.plot(x,norm.pdf(x,m1,S1),alpha=0.5,linewidth = 4)
plt.plot(x,norm.pdf(x,m2,S2),alpha=0.5,linewidth = 4)
plt.hist(Xp1, bins='auto', alpha=0.5, normed = 1,color='blue')
plt.hist(Xp2, bins='auto', alpha=0.5, normed = 1,color='green')
plt.show()